- 문서에 등장하는 각 단어의 등장빈도를 행렬로 표현한다.
- term = 단어, 
- 가로가 단어/ 세로가 문서인 행렬을 DTM
- 세로가 단어/ 가로가 문서인 행렬의 경우 TDM

# DTM 
- 가로가 단어(토큰) 세로가 문서 -> 행렬
- 단어 빈도수가 중요도를 의미하지는 않는다. -> 이를 보완하기 위해 TF - IDF를 사용한다.

# TDM

In [6]:
docs = ['동물원 코끼리',
       '동물원 원숭이 바나나',
       '엄마 코끼리 아기 코끼리',
       '원숭이 바나나 코끼리 바나나']

doc_ls = []
for doc in docs:
    doc_ls.append(doc.split())
doc_ls

[['동물원', '코끼리'],
 ['동물원', '원숭이', '바나나'],
 ['엄마', '코끼리', '아기', '코끼리'],
 ['원숭이', '바나나', '코끼리', '바나나']]

In [59]:
from collections import defaultdict
import numpy as np

word2dic = defaultdict(lambda :len(word2dic))


for lis in doc_ls:
    for token in lis:
        word2dic[token]
print(word2dic)
# 행이 단어의 갯수만큼, 열이 문서의 갯수만큼          

TDM = np.zeros((len(word2dic), len(doc_ls)), dtype = int)               
for i, words in enumerate(doc_ls):       
    for token in words:
        #### 행렬  #####
        TDM[word2dic[token],i] += 1
print(TDM)

defaultdict(<function <lambda> at 0x0000019EDBAD8C18>, {'동물원': 0, '코끼리': 1, '원숭이': 2, '바나나': 3, '엄마': 4, '아기': 5})
[[1 1 0 0]
 [1 0 2 1]
 [0 1 0 1]
 [0 1 0 2]
 [0 0 1 0]
 [0 0 1 0]]


In [60]:
import pandas as pd

doc_names = ['문서' + str(i+1) for i in range(len(doc_ls))]
print('doc_names', doc_names)

# sorted 순서대로 정렬한 리스트를 반환한다. 
# items()   dictionary의 key와 values를 쌍으로 Return.한다.
sorted_vocab = sorted((value, key) for key,value in word2dic.items())
vocab = [v[1] for v in sorted_vocab]
df_TDM = pd.DataFrame(TDM, columns = doc_names)
df_TDM['단어'] = vocab
df_TDM.set_index('단어')

doc_names ['문서1', '문서2', '문서3', '문서4']


,문서1,문서2,문서3,문서4
단어,,,,
동물원,1,1,0,0
코끼리,1,0,2,1
원숭이,0,1,0,1
바나나,0,1,0,2
엄마,0,0,1,0
아기,0,0,1,0


# -------------------------패키지사용 ----------------------------------------------------------------

# sklearn

In [38]:
# count_vect 토큰의 갯수를 세어줌

from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
DTM = count_vect.fit_transform(docs)
DTM.toarray()

array([[1, 0, 0, 0, 0, 1],
       [1, 1, 0, 0, 1, 0],
       [0, 0, 1, 1, 0, 2],
       [0, 2, 0, 0, 1, 1]], dtype=int64)

In [41]:
DTM.toarray().T

array([[1, 1, 0, 0],
       [0, 1, 0, 2],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 1, 0, 1],
       [1, 0, 2, 1]], dtype=int64)

In [ ]:
import pandas as pd

doc_names = ['문서' + str(i+1) for i in range(len(doc_ls))]
print('doc_names', doc_names)

sorted-vocab = sorted((key,values) for key,values in enumerate(count_vect.items()))


# gensim

In [42]:
import gensim
from gensim import corpora

doc_ls = [doc.split() for doc in docs]
id2word = corpora.Dictionary(doc_ls)
TDM = [id2word.doc2bow(doc) for doc in doc_ls]
TDM

[[(0, 1), (1, 1)],
 [(0, 1), (2, 1), (3, 1)],
 [(1, 2), (4, 1), (5, 1)],
 [(1, 1), (2, 2), (3, 1)]]

- 0번쨰, 1번째에 1을 넣고 나머지는 0
- 0,2,3번째에 1넣고 나머지는 0
- 1 인덱스는 2, 4와 5에는 1넣기 나머지는 0
- 1,3번째 인덱스는 1 2번쨰 인덱스는 2넣고 나머지는 0

## gensim -> 데이터프레임

In [48]:
from gensim .matutils import sparse2full
import pandas as pd
import numpy as np

doc_names = ['문서' + str(i) for i in range(len(doc_ls))]
vocab = [id2word[i] for i in  id2word.keys()]
DTM_matrix = [sparse2full(doc, len(vocab)).tolist() for doc in TDM]

df_TDM = pd.DataFrame(np.array(DTM_matrix, dtype= int).T, columns = doc_names)
df_TDM['단어'] = vocab
df_TDM.set_index('단어')

,문서0,문서1,문서2,문서3
단어,,,,
동물원,1,1,0,0
코끼리,1,0,2,1
바나나,0,1,0,2
원숭이,0,1,0,1
아기,0,0,1,0
엄마,0,0,1,0


# -------------------------------------------------------------------------------------------

# TF  - IDF
- TF : 단어 빈도수
- IDF : 문서의 역 빈도수 
- 단어의 상대적 중요도를 반영할 수 있다.
- TF * IDF 

TF
- 문서에 특정 단어가 등장하는 빈도수
- 문서내에 많이 단어가 등장할 수록 높은 TF값을 가진다. 자주 등장할 수록 중요한 단어라는 의미를 갖는다.
- 문서에 '인공지능'이 몇번 등장하는지

IDF
- 특정 단어가 등장한 문서의 수
- '인공지능'이라는 단어가 몇개의 문서에 등장하는지(DF)....의 역수
-  단어가 여러문서에 많이 등장할 수록 IDF는 작아진다.
- IDF에 로그를 사용하는 이유 , 값이 커질수록 수렴하기 위해서

- TDM내의 단어의 중요성을 가중치로 표현한 방법
- TDM을 사용하는 것보다 더 정확하게 문서비교가 가능하다.


- TF = 특정단어등장빈도/ 문서내 전체 등장 단어 빈도
- IDF = log(총문서수/ 단어가 등장한 문서수)
- TF-IDF = TF * IDF
- TF-IDF값이 클수록 중요단어

## TF-IDF 직접구현

In [63]:
docs = ['오늘 동물원에서 원숭이와 코끼리를 봤어',
        '동물원에서 원숭이에게 바나나를 줬어 바나나를']

In [64]:
doc_ls = []
for doc in docs:
    doc_ls.append(doc.split())
doc_ls

[['오늘', '동물원에서', '원숭이와', '코끼리를', '봤어'],
 ['동물원에서', '원숭이에게', '바나나를', '줬어', '바나나를']]

In [209]:
from collections import defaultdict
import numpy as np

word2id = defaultdict(lambda :len(word2id))

for lis in doc_ls:
    for token in lis:
        word2id[token]
print(len(word2id))

TDM = np.zeros((len(word2id), len(doc_ls)),dtype = int)

for i, words in enumerate(doc_ls):
    for token in words:
        TDM[word2id[token],i] += 1
        
TDM = TDM.T
TDM




8


array([[1, 1, 1, 1, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 1, 2, 1]])

In [237]:
def TF():
    per = []
    
    for lis in TDM:
        for val in lis:
            per.append(val/len(word2id))                    
    result = per[:7],per[8:16]
    return result

In [238]:
TF()

([0.125, 0.125, 0.125, 0.125, 0.125, 0.0, 0.0],
 [0.0, 0.125, 0.0, 0.0, 0.0, 0.125, 0.25, 0.125])

In [229]:
#word2id의 값이 둘다

def IDF():
    import math
    count = 0
    result = []
    for i in range(len(word2id)):                   
        if TDM[0][i] != 0  and TDM[1][i] != 0:
            count += 2
        elif TDM[0][i] != 0  or TDM[1][i] != 0:
            count += 1
        else:
            count += 0
        result.append(math.log10(2/count))
    np.array(result)
    return result

In [230]:
IDF()

[0.3010299956639812,
 -0.17609125905568127,
 -0.3010299956639812,
 -0.3979400086720376,
 -0.47712125471966244,
 -0.5440680443502757,
 -0.6020599913279624,
 -0.6532125137753437]

# -------------------------------------------------------------------------------------------

# TFIDF 직접 구현 실습코드

In [239]:
docs = ['오늘 동물원에서 원숭이와 코끼리를 봤어',
        '동물원에서 원숭이에게 바나나를 줬어 바나나를']

In [240]:
# 토큰화 하기
doc_ls = []
for doc in docs:
    doc_ls.append(doc.split())
    
    
doc_ls

[['오늘', '동물원에서', '원숭이와', '코끼리를', '봤어'],
 ['동물원에서', '원숭이에게', '바나나를', '줬어', '바나나를']]

In [241]:
from collections import defaultdict

word2id = defaultdict(lambda : len(word2id))
for doc in doc_ls:
    for token in doc:
        word2id[token]
word2id

defaultdict(<function __main__.<lambda>()>,
            {'오늘': 0,
             '동물원에서': 1,
             '원숭이와': 2,
             '코끼리를': 3,
             '봤어': 4,
             '원숭이에게': 5,
             '바나나를': 6,
             '줬어': 7})

In [243]:
import numpy as np

TDM = np.zeros((len(doc_ls),len(word2id)), dtype = int) #정수 자료형으로 출력

for i, doc in enumerate(doc_ls):
    for token in doc:
        # 행렬 만들기
        TDM[i,word2id[token]] += 1
TDM        

array([[1, 1, 1, 1, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 1, 2, 1]])

- TDM[i,word2id[token]] += 1
- token이 반복문을 돌면서 '오늘'이면 
- TDM[0,word2id['오늘]] = TDM[0,0]에 +1이 됨

In [244]:
# 1문서 토큰이 5개 -> sum으로 표현
TDM[0].sum()

5

TF = 특정단어등장빈도/ 문서내 전체 등장 단어 빈도

In [248]:
def computeTF(TDM):
    doc_len = len(TDM)      #문서 갯수 2개
    word_len = len(TDM[0])  # 토큰의 갯수 8개
    # tf를 계산하기 전 0으로 셋팅
    tf = np.zeros((doc_len,word_len))  # 문서갯수,토큰갯수
    print(tf)
    
    for doc_i in range(doc_len):
        for word_i in range(word_len):
            tf[doc_i, word_i] = TDM[doc_i, word_i]/TDM[doc_i].sum()
    return tf

In [249]:
computeTF(TDM)

[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]


array([[0.2, 0.2, 0.2, 0.2, 0.2, 0. , 0. , 0. ],
       [0. , 0.2, 0. , 0. , 0. , 0.2, 0.4, 0.2]])

 IDF계산 : log(총문서수/단어가등장한문서수)

In [250]:
import math 

def computeIDF(TDM):
    doc_len = len(TDM)
    word_len = len(TDM[0])
    
    idf = np.zeros(word_len)
    
    for i in range(word_len):
        idf[i] = math.log10(doc_len/ np.count_nonzero(TDM[:,i]))
    return idf


In [253]:
computeIDF(TDM)

array([0.30103, 0.     , 0.30103, 0.30103, 0.30103, 0.30103, 0.30103,
       0.30103])

In [292]:
tf = computeTF(TDM)
tf.shape[0]

[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]


2

In [274]:
# TF-IDF 곱

def computeTFIDF(TDM):
    tf = computeTF(TDM)
    idf = computeIDF(TDM)
    # tf shape대로 zero 행렬만들기
    tfidf = np.zeros(tf.shape)
    # 토큰 갯수만큼 for문
    # tf.shape[0] 행의 갯수, tf.shape[1] 열의 갯수 
    for doc_i in range(tf.shape[0]):
        for word_i in range(tf.shape[1]):
            tfidf[doc_i, word_i] = tf[doc_i, word_i] * idf[word_i]
    return tfidf


In [276]:
computeTFIDF(TDM)

[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]


array([[0.060206, 0.      , 0.060206, 0.060206, 0.060206, 0.      ,
        0.      , 0.      ],
       [0.      , 0.      , 0.      , 0.      , 0.      , 0.060206,
        0.120412, 0.060206]])

In [289]:
import pandas as pd

# 딕셔너리 -> 리스트 (키+값 쌍)
sorted_vocab = sorted((value, key) for key, value in word2id.items())
print(sorted_vocab)
# vocab == 토큰
vocab = [v[1] for v in sorted_vocab]

tfidf = computeTFIDF(TDM)
pd.DataFrame(tfidf,columns = vocab)

[(0, '오늘'), (1, '동물원에서'), (2, '원숭이와'), (3, '코끼리를'), (4, '봤어'), (5, '원숭이에게'), (6, '바나나를'), (7, '줬어')]
[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]


,오늘,동물원에서,원숭이와,코끼리를,봤어,원숭이에게,바나나를,줬어
0,0.060206,0.0,0.060206,0.060206,0.060206,0.000000,0.000000,0.000000
1,0.000000,0.0,0.000000,0.000000,0.000000,0.060206,0.120412,0.060206


# -------------------------------------------------------------------------------------------------

# 패키지사용

## sklearn

In [300]:
docs = ['오늘 동물원에서 원숭이와 코끼리를 봤어',
       '동물원에서 원숭이에게 바나나를 줬어 바나나를']

from sklearn.feature_extraction.text import TfidfVectorizer
tfidv = TfidfVectorizer()
tfidv = tfidv.fit(docs)
tfidv.transform(docs).toarray()
vocab = tfidv.get_feature_names()

In [301]:
import pandas as pd
df = pd.DataFrame(tfidv.transform(docs).toarray(), columns = vocab)
df

,동물원에서,바나나를,봤어,오늘,원숭이에게,원숭이와,줬어,코끼리를
0,0.335176,0.000000,0.471078,0.471078,0.000000,0.471078,0.000000,0.471078
1,0.278943,0.784088,0.000000,0.000000,0.392044,0.000000,0.392044,0.000000


## gensim

In [304]:
import gensim
from gensim import corpora
from gensim.models import TfidfModel

docs = ['오늘 동물원에서 원숭이와 코끼리를 봤어',
       '동물원에서 원숭이에게 바나나를 줬어 바나나를']

doc_ls = [doc.split() for doc in docs] #공백으로 토큰화
id2word = corpora.Dictionary(doc_ls)
print(id2word)

# TDM 생성
TDM = [id2word.doc2bow(doc) for doc in doc_ls]


model = TfidfModel(TDM)
tfidf = model[TDM]
tfidf[0]

Dictionary(8 unique tokens: ['동물원에서', '봤어', '오늘', '원숭이와', '코끼리를']...)


[(1, 0.5), (2, 0.5), (3, 0.5), (4, 0.5)]

In [305]:
import pandas as pd
from gensim.matutils import sparse2full

vocab = [id2word[i] for i in  id2word.keys()]
TDM_matrix = [sparse2full(doc, len(vocab)).tolist() for doc in tfidf]
pd.DataFrame(TDM_matrix, columns = vocab)

,동물원에서,봤어,오늘,원숭이와,코끼리를,바나나를,원숭이에게,줬어
0,0.0,0.5,0.5,0.5,0.5,0.000000,0.000000,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.816497,0.408248,0.408248
